# 异步IO-Selector

In [1]:
import socket
import selectors
from selectors import SelectorKey
from typing import Any

In [2]:
sel = selectors.DefaultSelector()

In [3]:
def read(sock: socket.socket, mask):
    data = sock.recv(1000)
    if data:
        print(f'receive msg: {data.decode()}, mask: {mask}')
        sock.send(data)
    else:
        print(f'closing {sock}')
        sel.unregister(conn)
        sock.close()

In [4]:
def accept(sock: socket.socket, mask):
    conn, addr = sock.accept()
    print(f'accept connection from {addr}, mask: {mask}')
    conn.setblocking(False)
    sel.register(conn, selectors.EVENT_READ, read)

In [5]:
def main():
    sock = socket.socket()
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    addr = ('localhost', 4444)
    sock.bind(addr)
    sock.listen(100)
    sock.setblocking(False)
    print(f'listen address: {addr}, and waiting connection.')

    sel.register(sock, selectors.EVENT_READ, accept)

    while True:
        print('selecting...')
        events = sel.select()
        for key, mask in events:
            callback = key.data
            callback(key.fileobj, mask)